### Fitting

In physics, the measurement is mainly for two things. 
+ Testing hypothesis
+ Parameter setting

Dealing with testing hypothesis is beyond our scope we are aiming for in this class. 
We will focus on how we do the parameter setting using data from experiments. 

In parameter setting, there are two ways to do this. 
+ Likelihood fitting
+ Least squares fitting

We will have a chance to talk about the Likelihood fitting in another place. In this exercise, we will use Least squres fitting which is defined as follows:

$$ \chi^2(\theta) = \sum^N_{i=1} \frac{(y_i - \lambda(x_i;\theta))^2}{\sigma_i^2} $$

$\theta$ is the parameter we would like to measure. 
$y_i$ is the result value from each measurement. $\sigma_i$ is the uncertainty from each measurement.
$x_i$ is the variable for the fit function of $\lambda$. <b>To determine the parameter $\theta$, we minimize this quantity $\chi^2$</b>. 

Fortunately we don't need to implement the $\chi$ ourselves everytime we determine the parameter. We will use the class implemented in ROOT data analysis framework which was developed at CERN to deal with big data. 

In [ ]:
#we will rely on ROOT data analysis framework for fitting
import ROOT

#### Reading txt file

Firstly, let's assume that we have following measurements. We will save this information into txt file, `input_expected.txt`
```
# Measurement of Friday 26 March
# Experiment 2 Physics Lab

1   6   5
2   12  5
3   14  4.7
4   20  4.5
5   22  4.2
6   24  5.1
7   35  2.9
8   45  4.1
9   44  4.8
10  53  5.43
```

In [ ]:
from ROOT import TF1, TH1F, TCanvas, TGraphErrors

graph_expected = TGraphErrors("./input_expected.txt","%lg %lg %lg")

In [ ]:
graph_expected.SetTitle(
       "Measurement XYZ and Expectation;"
       "lenght [cm];"
       "Arb.Units")
#graph_expected.SetMakerStyle(20) #yello
#graph_expected.DrawClone("PA") # E3 draws the band
c = TCanvas()
graph_expected.Draw("PA*")
c.Draw()

#### Fitting with linear function

In [ ]:
class Linear:
    def __call__( self, x, par ):
        return par[0] + x[0]*par[1]

# create a linear function with offset 5, and pitch 2
pyf1 = Linear()
f = TF1('pyf1',pyf1,-1.,1.,2)
f.SetParameters(5.,5.) # with these parameters, the function becomes y = 5+2x 

In [ ]:
graph_expected.Fit(f)
# print results
par = f.GetParameters()
print ('fit results: const =', par[0], ',pitch =', par[1])

In [ ]:
#c = TCanvas()
graph_expected.Draw("PA")
c.Draw()

### Fitting to dimuon mass

Gaussian function : $$ G(x;\mu,\sigma) = \frac{N}{\sqrt{2 \pi \sigma}} \exp{ \big[ \frac{-(x-\mu)^2}{ 2\sigma^2} \big] } $$

Relativistic Breit-Wigner : $$ B(m;M,\Gamma) = N \cdot \frac{2}{\pi} \cdot \frac{\Gamma^2M^2}{(m^2-M^2)^2+m^4(\Gamma^2/M^2)} $$

Convolution of relativistic Breit-Wigner plus a Gaussian (voigt) : $$ P(m;M,\Gamma,\sigma) = \int B(m^\prime;M,\Gamma) \cdot G(m-m^\prime;\mu, \sigma) dm^\prime $$ 

#### Fitting with Gaussian distribution

In [ ]:
import math
class Gaus:
    def __call__( self, x, par ):
        return par[0]/(math.sqrt(2.0*3.14151927)*par[2])*math.exp(-(x[0]-par[1])*(x[0]-par[1])/(2.0*par[2]*par[2]))

# create a Gaussian function with number of events of 1000, a mean of 91.2 GeV and 10 GeV resolution
f2 = TF1('pyf2',Gaus(),0.,200.,3)
f2.SetParameters(1000.,91.2,10) # with these parameters, the function becomes y = 5+2x 

Here we will use the ```output2.root``` file that was created in the ComPhy-3 section. 

In [ ]:
from ROOT import TChain, TFile

# open the file
chain = TChain("tree");
chain.Add("output2.root")

h_mass = TH1F('h_mass','dimuon mass',100,50.0,150.0)

entries = chain.GetEntries()
print ("total number of events = ", entries)
for i in range(entries):
    chain.GetEntry(i)
    h_mass.Fill(chain.dimuon_mass) 
    if i%100000 == 0:
        print ("muon mass = ", chain.dimuon_mass)
print ("done")
c = TCanvas()
h_mass.Draw()
c.Draw()

In [ ]:
#we will have to put the following line in to fit 
#h_mass.Fit(f2)
# print results
par = f2.GetParameters()
print ('fit results: number of events =', par[0], ', mean =', par[1], ' sigma = ', par[2])

In [ ]:
from ROOT import gStyle

c = TCanvas()
h_mass.Draw()
h_mass.SetStats(1) # draw window for stats information 
gStyle.SetOptStat(11) # print only name of histogram and number of entries.
gStyle.SetOptFit(111) # print chi2/number of degree of freedom, errors, name/values of parameters
c.Draw()

The number of degree of freedom is the number of bins subtracting the number of parameters.
In this example, 
```
ndf (number of degrees of freedom) = 100-3 = 97 
```
This value is close to 1 if the fitting performance is good. In this example, the ```chi2/ndf ~ 31``` which is far way from 1. So we can say the fitting is not really good as you can see by eyes. 

#### Fitting with Relativistic Breit-Wigner distribution

In [ ]:
mybw = TF1("mybw","[0]*TMath::BreitWigner(x, [1], [2])", 50, 150);
mybw.SetParName(0, "number of signal");
mybw.SetParName(1, "mean");
mybw.SetParName(2, "gamma");
mybw.SetParameters(5000.0, 91.2 , 2);

In [ ]:
h_mass2 = h_mass.Clone("h_mass2")
h_mass2.Fit(mybw)
# print results
par = mybw.GetParameters()
print ('fit results: number of events =', par[0], ', mean =', par[1], ' gamma = ', par[2])

In [ ]:
c = TCanvas()
h_mass2.Draw()
h_mass2.SetStats(1) # draw window for stats information 
gStyle.SetOptStat(11) # print only name of histogram and number of entries.
gStyle.SetOptFit(100) # print chi2/number of degree of freedom, errors, name/values of parameters
c.Draw()

```chi2/ndf = 7.6``` so it is getting better.

#### Fitting with Convolution of relativistic Breit-Wigner and Gaussian

In [ ]:
myvoigt = TF1("myvoigt","[0]*TMath::Voigt(x-[1], [2], [3])", 50, 150);
myvoigt.SetParName(0, "number of signal");
myvoigt.SetParName(1, "mean");
myvoigt.SetParName(2, "sigma");
myvoigt.SetParName(3, "gamma");
myvoigt.SetParameters(20000.0, 91.2 , 2.5 , 2);

In [ ]:
h_mass3 = h_mass.Clone("h_mass3")
h_mass3.Fit(myvoigt)
# print results
par = myvoigt.GetParameters()
print ('fit results: number of events =', par[0], ', mean =', par[1], ' sigma = ', par[2], ' gamma = ', par[3])

In [ ]:
c = TCanvas()
h_mass3.Draw()
h_mass3.SetStats(1) # draw window for stats information 
gStyle.SetOptStat(11) # print only name of histogram and number of entries.
gStyle.SetOptFit(100) # print chi2/number of degree of freedom, errors, name/values of parameters
c.Draw()

```chi2/ndf = 6.3``` so it is even getting closer to 1. 

What is the mass of Z boson and its width you measured from this fitting?

In [ ]:
print ("mean =", par[1], "width = ", par[3])

##### answer: 
```
mean = 90.734433871  width =  3.32081965198
```

#### Fitting with background (exponential function)

In [ ]:
# we add exponential function of a*exp(b+cx) in the fitting
myvoigt_bkg = TF1("myvoigt","[0]*TMath::Voigt(x-[1], [2], [3])+[4]*TMath::Exp([5]+[6]*(x-[1]))", 50, 150)
myvoigt_bkg.SetParName(0, "number of signal")
myvoigt_bkg.SetParName(1, "mean")
myvoigt_bkg.SetParName(2, "sigma")
myvoigt_bkg.SetParName(3, "gamma")
myvoigt_bkg.SetParName(4, "a")
myvoigt_bkg.SetParName(5, "b")
myvoigt_bkg.SetParName(6, "c")
myvoigt_bkg.SetParameters(20000.0, 91.2 , 2.5 , 2)

h_mass4 = h_mass.Clone("h_mass4")
h_mass4.Fit(myvoigt_bkg)
# print results
par = myvoigt_bkg.GetParameters()
print ('fit results: ')
print ('number of events =', par[0], ', mean =', par[1], ' sigma = ', par[2], ' gamma = ', par[3])
print ('a = ', par[4], ', b =', par[5], ', c = ', par[6])

In [ ]:
c = TCanvas()
h_mass4.Draw()
h_mass4.SetStats(1) # draw window for stats information 
gStyle.SetOptStat(11) # print only name of histogram and number of entries.
gStyle.SetOptFit(100) # print chi2/number of degree of freedom, errors, name/values of parameters
c.Draw()

Check the chi2/ndf.
```
chi2/ndf = 252.9/93 = 2.7
```
Not it is getting more closer to 1 

How about the mean and width of the Z boson?

In [ ]:
print ("mean =", par[1], "width = ", par[3])

##### answer:
```
mean = 90.7545783801 width =  2.97194636941
```

More details about the Z boson can be found here.  
<http://pdg.lbl.gov/2018/listings/rpp2018-list-z-boson.pdf>

In this reference, you can see that the width of Z boson is 2.5 GeV.

Can you think of another way to improve your measurement?